In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import random
import implicit

pd.set_option('display.max_rows', 100)

import warnings
warnings.filterwarnings("ignore")

In [2]:
users = pd.read_csv("rec_sys/user.csv")
print(users.shape)
users.head()

(1973, 4)


,AnonId,Sessionid,Country,City
0,97293cdc-6098-4ffc-83fc-221166595a27,de8a36c6-a59d-454e-813a-10b410d232a4,RS,Pančevo
1,ac199577-1638-40fe-b0d9-c82806d9995e,8511cb18-2b11-4ff9-a0cc-38872e44f88e,RU,Moscow
2,8efccb37-8a6b-4e72-82bf-6b7b7e896ca9,39844256-5af9-4482-b1d9-6ecadc694316,RS,NaN
3,2f4b2f56-0c22-4486-a9e5-fc76ab77820e,613b2399-8d3a-43a5-a53f-337077c6d828,RS,Smederevo
4,a908ff62-fe6d-4201-9a70-5474f4938a52,6ef9e025-b843-4d94-b49c-65a60b2a45ac,RS,Belgrade


In [3]:
activities = pd.read_csv("rec_sys/activity.csv")
print(activities.shape)
activities.head()

(4704, 3)


,AnonId,HotelName,HotelCategory
0,97293cdc-6098-4ffc-83fc-221166595a27,CAVALIERI HOTEL,4
1,458b7809-dec1-4948-958b-4c085173fffa,SAPHIR HOTEL,4
2,6b85379a-1d99-427f-a931-b311bfae472b,ASPALACE HOTEL THE ISTANBUL OLD CITY,3
3,8efccb37-8a6b-4e72-82bf-6b7b7e896ca9,AMARA DOLCE VITA LUXURY,5
4,2f4b2f56-0c22-4486-a9e5-fc76ab77820e,NAVARRIA HOTEL,3


In [4]:
hotels = pd.read_csv("rec_sys/hotel.csv")
print(hotels.shape)
hotels.head()

(1208, 2)


,HotelName,HotelCategory
0,KATYA BEACH HOTEL,5
1,VIHREN PALACE,4
2,HOTEL GOLF PRAGUE,4
3,PENSION BOHMERWALD,3
4,APOLLON HOTEL,4


# preprocess

In [5]:
def missing(df):
    miss = df.isnull().sum()
    return miss[miss > 0]

In [6]:
missing(users)

AnonId         1
Sessionid      3
Country        2
City         275
dtype: int64

In [7]:
users.dropna(subset=["AnonId", "Sessionid", "Country"], axis=0, inplace=True)

In [8]:
users.groupby("Country")["City"].apply(lambda x: pd.Series.mode(x).values)

Country
AL                                                   []
AT                                             [Vienna]
BA                                           [Sarajevo]
BE                                           [Verviers]
BG                                              [Sofia]
CH                        [Duillier, Guensberg, Zurich]
DE    [Berlin, Bielefeld, Frankfurt am Main, Munich,...
EG                                              [Cairo]
ES                                           [Albacete]
FI                                           [Helsinki]
FR                                             [Pantin]
GB                                                   []
GE                                            [Tbilisi]
HR                                             [Zagreb]
HU                                           [Budapest]
IE                                 [Dublin, Inishannon]
IQ                                       [Sulaymaniyah]
IT                                     [

In [9]:
users[users["Country"]=="AL"] = users[users["Country"]=="AL"].fillna("AL")
users[users["Country"]=="GB"] = users[users["Country"]=="GB"].fillna("GB")

In [10]:
city_na = users.groupby("Country")["City"].apply(lambda x: pd.Series.mode(x).values[0]).to_dict()
users["City"] = users["Country"].map(city_na)

missing(users)

Series([], dtype: int64)

In [11]:
missing(activities)

AnonId    1
dtype: int64

In [12]:
activities.dropna(inplace=True)
missing(activities)

Series([], dtype: int64)

In [13]:
missing(hotels)

Series([], dtype: int64)

In [14]:
df = users.drop("Sessionid", axis=1).merge(activities, on="AnonId").drop_duplicates()
df

,AnonId,Country,City,HotelName,HotelCategory
0,97293cdc-6098-4ffc-83fc-221166595a27,RS,Belgrade,CAVALIERI HOTEL,4
1,97293cdc-6098-4ffc-83fc-221166595a27,RS,Belgrade,ARMADA BLUE BAY,4
2,ac199577-1638-40fe-b0d9-c82806d9995e,RU,Moscow,SUN BEACH HOTEL,4
3,ac199577-1638-40fe-b0d9-c82806d9995e,RU,Moscow,ARISTOTELES HOLIDAY RESORT & SPA (BOMO),4
4,ac199577-1638-40fe-b0d9-c82806d9995e,RU,Moscow,SANTA BEACH HOTEL,4
...,...,...,...,...,...
8523,ec697358-765c-4729-a1fe-0855aedea0cd,RS,Belgrade,MAYA WORLD BEACH ALANYA,4
8524,ec697358-765c-4729-a1fe-0855aedea0cd,RS,Belgrade,SAPHIR HOTEL,4
8525,808c30a0-4c1f-466a-841e-2c5c67186e8f,RS,Belgrade,SULTAN SIPAHI RESORT,4
8526,dbca78c1-dd3a-4cac-8ed6-13bfc647cd5b,RS,Belgrade,GRAND OASIS RESORT,4


In [15]:
df.HotelCategory.value_counts()

5         1912
4         1825
3          706
0          127
2           95
4Stars       4
5Stars       1
3Stars       1
Name: HotelCategory, dtype: int64

In [16]:
map_hotel = {"3Stars": 3, "4Stars": 4, "5Stars":5}
df["HotelCategory"].replace(map_hotel, inplace=True)
df["HotelCategory"] = df["HotelCategory"].astype(int)

In [17]:
grouped = df.groupby(["AnonId", "Country", "City", "HotelName"]).sum().reset_index()
grouped

,AnonId,Country,City,HotelName,HotelCategory
0,0000d271-8b7e-4bb6-bc06-ffbe9412864a,RS,Belgrade,AL KHOORY ATRIUM,4
1,0000d271-8b7e-4bb6-bc06-ffbe9412864a,RS,Belgrade,ARMADA BLUE BAY,4
2,0000d271-8b7e-4bb6-bc06-ffbe9412864a,RS,Belgrade,DUKES HOTEL,5
3,0000d271-8b7e-4bb6-bc06-ffbe9412864a,RS,Belgrade,GRAND EXCELSIOR BUR DUBAI,4
4,0000d271-8b7e-4bb6-bc06-ffbe9412864a,RS,Belgrade,JONRAD HOTEL,3
...,...,...,...,...,...
4661,ff151134-9044-4867-abbb-aa41bd8310a5,RS,Belgrade,SHARM GRAND PLAZA,5
4662,ff3be02c-3e46-4f35-8f46-094770638159,RS,Belgrade,EDEN HOTEL,4
4663,ff3be02c-3e46-4f35-8f46-094770638159,RS,Belgrade,VIENNA HOTEL,3
4664,ff620c52-bbfa-4594-8906-7bb57cd59f15,RS,Belgrade,IC HOTELS GREEN PALACE,5


In [18]:
grouped["AnonId"]=grouped["AnonId"].astype("category")
grouped["Country"]=grouped["Country"].astype("category")
grouped["City"]=grouped["City"].astype("category")
grouped["HotelName"]=grouped["HotelName"].astype("category")

grouped["Anon_id"] = grouped["AnonId"].cat.codes
grouped["Country_id"] = grouped["Country"].cat.codes
grouped["City_id"] = grouped["City"].cat.codes
grouped["Hotel_id"] = grouped["HotelName"].cat.codes

In [19]:
grouped

,AnonId,Country,City,HotelName,HotelCategory,Anon_id,Country_id,City_id,Hotel_id
0,0000d271-8b7e-4bb6-bc06-ffbe9412864a,RS,Belgrade,AL KHOORY ATRIUM,4,0,24,2,15
1,0000d271-8b7e-4bb6-bc06-ffbe9412864a,RS,Belgrade,ARMADA BLUE BAY,4,0,24,2,63
2,0000d271-8b7e-4bb6-bc06-ffbe9412864a,RS,Belgrade,DUKES HOTEL,5,0,24,2,230
3,0000d271-8b7e-4bb6-bc06-ffbe9412864a,RS,Belgrade,GRAND EXCELSIOR BUR DUBAI,4,0,24,2,315
4,0000d271-8b7e-4bb6-bc06-ffbe9412864a,RS,Belgrade,JONRAD HOTEL,3,0,24,2,405
...,...,...,...,...,...,...,...,...,...
4661,ff151134-9044-4867-abbb-aa41bd8310a5,RS,Belgrade,SHARM GRAND PLAZA,5,1581,24,2,762
4662,ff3be02c-3e46-4f35-8f46-094770638159,RS,Belgrade,EDEN HOTEL,4,1582,24,2,233
4663,ff3be02c-3e46-4f35-8f46-094770638159,RS,Belgrade,VIENNA HOTEL,3,1582,24,2,869
4664,ff620c52-bbfa-4594-8906-7bb57cd59f15,RS,Belgrade,IC HOTELS GREEN PALACE,5,1583,24,2,381


In [20]:
anon_country = sparse.csr_matrix((grouped['HotelCategory'].astype(float), (grouped["Anon_id"], grouped['Country_id'])))

city_anon = sparse.csr_matrix((grouped['HotelCategory'].astype(float), (grouped["City_id"], grouped['Anon_id'])))

anon_hotel = sparse.csr_matrix((grouped['HotelCategory'].astype(float), (grouped["Anon_id"], grouped['Hotel_id'])))

hotel_anon = sparse.csr_matrix((grouped['HotelCategory'].astype(float), (grouped["Hotel_id"], grouped['Anon_id'])))

data_in = hotel_anon * anon_country * city_anon
data_out = anon_country * city_anon * anon_hotel

In [21]:
data_in

<915x1584 sparse matrix of type '<class 'numpy.float64'>'
	with 157974 stored elements in Compressed Sparse Row format>

In [22]:
1- 157974/(915*1584) #sparsity

0.8910043053485677

In [23]:
ALS = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.01, iterations=100, calculate_training_loss=True)

In [24]:
alpha = 0.15
ALS.fit(data_in*alpha)

# User & Item Similarities / ALS model 

In [25]:
ALS.similar_users(42)

[(42, 10.982211),
 (113, 10.979881),
 (511, 10.975535),
 (734, 2.762984),
 (1396, 2.755413),
 (30, 2.7507405),
 (539, 2.7408948),
 (905, 2.7380414),
 (1023, 2.349169),
 (184, 2.1955407)]

In [26]:
grouped[grouped["Anon_id"] == 42]

,AnonId,Country,City,HotelName,HotelCategory,Anon_id,Country_id,City_id,Hotel_id
99,0612d74a-a20c-4f80-bb8f-55ce38f0a45b,TR,Istanbul,MIRAGE PARK RESORT,5,42,28,12,534


In [27]:
grouped[grouped["Anon_id"] == 113]

,AnonId,Country,City,HotelName,HotelCategory,Anon_id,Country_id,City_id,Hotel_id
327,0f8e8785-5428-4522-bdfa-dda3a4e7ff9e,TR,Istanbul,JONRAD HOTEL,3,113,28,12,405


In [28]:
ALS.similar_items(553)

[(553, 0.099544056),
 (717, 0.08660391),
 (334, 0.08391759),
 (357, 0.083334975),
 (495, 0.08326946),
 (776, 0.083245836),
 (474, 0.08321289),
 (769, 0.083164774),
 (447, 0.083138034),
 (676, 0.082858644)]

In [29]:
grouped[grouped["Hotel_id"] == 553]

,AnonId,Country,City,HotelName,HotelCategory,Anon_id,Country_id,City_id,Hotel_id
3559,c0be7973-059f-4f86-bf37-e7ff6ddd6cd7,BA,Sarajevo,MPM GUINNESS HOTEL,4,1204,2,22,553
4119,de1cd9a1-80e4-4985-9d0e-eace0bdca8a9,MK,Skopje,MPM GUINNESS HOTEL,4,1396,19,23,553
4301,eafdec22-98da-40a0-af51-1145fc20669e,RS,Belgrade,MPM GUINNESS HOTEL,4,1466,24,2,553


In [30]:
grouped[grouped["Hotel_id"] == 717]

,AnonId,Country,City,HotelName,HotelCategory,Anon_id,Country_id,City_id,Hotel_id
2128,7538d028-4bfa-4472-b914-e213133b7190,MK,Skopje,SAILY HOTEL,3,734,19,23,717


In [31]:
ALS.recommend(767, data_out, filter_already_liked_items=True)

[(358, 0.9995059),
 (105, 0.99921775),
 (241, 0.005210325),
 (659, 0.0014193058),
 (547, 0.0013900697),
 (370, 0.0013190582),
 (10, 0.0013037771),
 (245, 0.0009741485),
 (12, 0.0008798018),
 (682, 0.0006994717)]

In [32]:
grouped[grouped.Anon_id == 767]

,AnonId,Country,City,HotelName,HotelCategory,Anon_id,Country_id,City_id,Hotel_id
2240,7afdd4cb-7def-4646-b068-54c5ff4f6f76,BA,Sarajevo,ELISABETHPARK,4,767,2,22,245


In [33]:
grouped[grouped.Hotel_id == 358]

,AnonId,Country,City,HotelName,HotelCategory,Anon_id,Country_id,City_id,Hotel_id
3565,c0edac2c-3085-45fa-9419-72e64029f65c,PT,Porto,HM BALANGUERA BEACH - PLAYA DE PALMA,4,1205,22,21,358


In [34]:
recs = ALS.recommend_all(data_out, N=1).tolist()

In [35]:
len(pd.Series(recs).value_counts()) #1584 kullanıcı için toplam 45 farklı otel önerisi  XXX

45

In [36]:
#A Gentle Introduction to Recommender Systems with Implicit Feedback
#https://jessesw.com/Rec-System/

In [37]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds))

In [38]:
train, test, users_ = make_train(data_in, pct_test = 0.2)

In [39]:
train

<915x1584 sparse matrix of type '<class 'numpy.float64'>'
	with 126379 stored elements in Compressed Sparse Row format>

In [40]:
def implicit_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 10, rank_size = 20, seed = 0):
    '''
    Implicit weighted ALS taken from Hu, Koren, and Volinsky 2008. Designed for alternating least squares and implicit
    feedback based collaborative filtering. 
    
    parameters:
    
    training_set - Our matrix of ratings with shape m x n, where m is the number of users and n is the number of items.
    Should be a sparse csr matrix to save space. 
    
    lambda_val - Used for regularization during alternating least squares. Increasing this value may increase bias
    but decrease variance. Default is 0.1. 
    
    alpha - The parameter associated with the confidence matrix discussed in the paper, where Cui = 1 + alpha*Rui. 
    The paper found a default of 40 most effective. Decreasing this will decrease the variability in confidence between
    various ratings.
    
    iterations - The number of times to alternate between both user feature vector and item feature vector in
    alternating least squares. More iterations will allow better convergence at the cost of increased computation. 
    The authors found 10 iterations was sufficient, but more may be required to converge. 
    
    rank_size - The number of latent features in the user/item feature vectors. The paper recommends varying this 
    between 20-200. Increasing the number of features may overfit but could reduce bias. 
    
    seed - Set the seed for reproducible results
    
    returns:
    
    The feature vectors for users and items. The dot product of these feature vectors should give you the expected 
    "rating" at each point in your original matrix. 
    '''
    
    # first set up our confidence matrix
    
    conf = (alpha*training_set) # To allow the matrix to stay sparse, I will add one later when each row is taken 
                                # and converted to dense. 
    num_user = conf.shape[0]
    num_item = conf.shape[1] # Get the size of our original ratings matrix, m x n
    
    # initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size))) # Random numbers in a m x rank shape
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size))) # Normally this would be rank x n but we can 
                                                                 # transpose at the end. Makes calculation more simple.
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)
    lambda_eye = lambda_val * sparse.eye(rank_size) # Our regularization term lambda*I. 
    
    # We can compute this before iteration starts. 
    
    # Begin iterations
   
    for iter_step in range(iterations): # Iterate back and forth between solving X given fixed Y and vice versa
        # Compute yTy and xTx at beginning of each iteration to save computing time
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        # Being iteration to solve for X based on fixed Y
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() # Grab user row from confidence matrix and convert to dense
            pref = conf_samp.copy() 
            pref[pref != 0] = 1 # Create binarized preference vector 
            CuI = sparse.diags(conf_samp, [0]) # Get Cu - I term, don't need to subtract 1 since we never added it 
            yTCuIY = Y.T.dot(CuI).dot(Y) # This is the yT(Cu-I)Y term 
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) # This is the yTCuPu term, where we add the eye back in
                                                      # Cu - I + I = Cu
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu) 
            # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu, equation 4 from the paper  
        # Begin iteration to solve for Y based on fixed X 
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray() # transpose to get it in row format and convert to dense
            pref = conf_samp.copy()
            pref[pref != 0] = 1 # Create binarized preference vector
            CiI = sparse.diags(conf_samp, [0]) # Get Ci - I term, don't need to subtract 1 since we never added it
            xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # This is the xTCiPi term
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
    # End iterations
    return X, Y.T # Transpose at the end to make up for not being transposed at the beginning. 
                         # Y needs to be rank x n. Keep these as separate matrices for scale reasons. 

In [41]:
user_vecs, item_vecs = implicit_weighted_ALS(train, lambda_val = 0.1, alpha = 15, iterations = 3,
                                            rank_size = 20)

In [42]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                          iterations = 50, 
                                                          calculate_training_loss=True)

In [43]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)   

In [44]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three 

In [45]:
calc_mean_auc(train, users_, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], test)
# AUC for our recommender system

(0.799, 0.787)

# LightFM model with City & Country

In [46]:
from lightfm.cross_validation import random_train_test_split

In [47]:
train, test = random_train_test_split(data_in, test_percentage=0.25)

In [48]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score

In [49]:
model = LightFM(loss='warp-kos')
%time model.fit(train, epochs=30, num_threads=4)

Wall time: 1min 20s


In [50]:
print("Train precision: %.2f" % precision_at_k(model, train, k=5).mean())
print("Test precision: %.2f" % precision_at_k(model, test, k=5).mean())

Train precision: 0.36
Test precision: 0.12


In [51]:
print("Train recall: %.2f" % recall_at_k(model, train, k=5).mean())
print("Test recall: %.2f" % recall_at_k(model, test, k=5).mean())

Train recall: 0.72
Test recall: 0.50


In [52]:
print("Train auc: %.2f" % auc_score(model, train).mean())
print("Test auc: %.2f" % auc_score(model, test).mean())

Train auc: 0.96
Test auc: 0.80


# Without  City & Country

In [53]:
train2, test2 = random_train_test_split(anon_hotel, test_percentage=0.25)

In [54]:
lfm_model2 = LightFM(loss="warp-kos")
%time lfm_model2.fit(train2, epochs=30)

Wall time: 1.58 s


In [55]:
print("Train precision: %.2f" % precision_at_k(lfm_model2, train2, k=5).mean())
print("Test precision: %.2f" % precision_at_k(lfm_model2, test2, k=5).mean())

Train precision: 0.20
Test precision: 0.01


In [56]:
print("Train recall: %.2f" % recall_at_k(lfm_model2, train2, k=5).mean())
print("Test recall: %.2f" % recall_at_k(lfm_model2, test2, k=5).mean())

Train recall: 0.58
Test recall: 0.04


In [57]:
print("Train auc: %.2f" % auc_score(lfm_model2, train2).mean())
print("Test auc: %.2f" % auc_score(lfm_model2, test2).mean())

Train auc: 0.98
Test auc: 0.69
